In [1]:
# 余分な Warning を非表示にする
import warnings

warnings.filterwarnings('ignore')

# Library の Import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# matplotlib 日本語化対応
import japanize_matplotlib

# 表示 Option の調整
np.set_printoptions(suppress=True, precision=4)  # NumPy の浮動小数点の表示精度
pd.options.display.float_format = '{:.4f}'.format  # pandas での浮動小数点の表示精度
pd.set_option('display.max_columns', None)  # DataFrame ですべての項目を表示
plt.rcParams['font.size'] = 14  # Graph の Default font 指定
random_seed = 123  # 乱数の種

In [2]:
# 追加 Library の import
import seaborn as sns

# Sample data の読み込み
df_titanic  = sns.load_dataset('titanic')

# 項目名の日本語化
columns = ['生存', '等室', '性別', '年齢', '兄弟配偶者数', '両親子供数', '料金', '乗船港 Code', '等室名', '男女子供', '成人男子', 'Deck', '乗船港', '生存可否', '独身']
df_titanic.columns = columns

# Data の内容
display(df_titanic.head())

,生存,等室,性別,年齢,兄弟配偶者数,両親子供数,料金,乗船港 Code,等室名,男女子供,成人男子,Deck,乗船港,生存可否,独身
0,0,3,male,22.0000,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0000,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0000,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0000,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0000,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [3]:
# 余分な列削除

# 「等室名」（「等室」との同じ）
df1 = df_titanic.drop('等室名', axis=1)

# 「乗船港」（「乗船港 Code」と同じ
df2 = df1.drop('乗船港', axis=1)

# 「生存可否」（「生存」と同じ）
df3 = df2.drop('生存可否', axis=1)

# 結果確認
display(df3.head())

,生存,等室,性別,年齢,兄弟配偶者数,両親子供数,料金,乗船港 Code,男女子供,成人男子,Deck,独身
0,0,3,male,22.0000,1,0,7.2500,S,man,True,NaN,False
1,1,1,female,38.0000,1,0,71.2833,C,woman,False,C,False
2,1,3,female,26.0000,0,0,7.9250,S,woman,False,NaN,True
3,1,1,female,35.0000,1,0,53.1000,S,woman,False,C,False
4,0,3,male,35.0000,0,0,8.0500,S,man,True,NaN,True


## 欠損値の対応

In [4]:
# 欠損値確認
print(df3.isnull().sum())

生存            0
等室            0
性別            0
年齢          177
兄弟配偶者数        0
両親子供数         0
料金            0
乗船港 Code      2
男女子供          0
成人男子          0
Deck        688
独身            0
dtype: int64


In [5]:
display(df3['Deck'].value_counts())

C    59
B    47
D    33
E    32
A    15
F    13
G     4
Name: Deck, dtype: int64

In [6]:
# 乗船港 Code: 欠損行数が２件と少ない
# -> 行ごと削除する

# dropna 関数を利用する
df4 = df3.dropna(subset=['乗船港 Code'])

# 年齢: 数値 Data であり欠損行数が177行とかなり多い
# -> 他 Data の平均値で代用

# 平均値の計算
age_average = df4['年齢'].mean()

# fillna 関数の利用
df5 = df4.fillna({'年齢': age_average})

# Deck: Label 値 Data であり欠損行数が688行と相当多い
# -> 欠損値を意味する Dummy code を振って全行を処理対象とする

# replace 関数の利用（Dummy code は'N'とする Deck 項目を Category 型から str 型に変更する）
df5['Deck'] = df5['Deck'].astype(np.str)
df6 = df5.replace({'Deck': {'nan': 'N'}})

In [7]:
# 結果確認
print(df6.isnull().sum())

display(df6.head())

生存          0
等室          0
性別          0
年齢          0
兄弟配偶者数      0
両親子供数       0
料金          0
乗船港 Code    0
男女子供        0
成人男子        0
Deck        0
独身          0
dtype: int64


,生存,等室,性別,年齢,兄弟配偶者数,両親子供数,料金,乗船港 Code,男女子供,成人男子,Deck,独身
0,0,3,male,22.0000,1,0,7.2500,S,man,True,N,False
1,1,1,female,38.0000,1,0,71.2833,C,woman,False,C,False
2,1,3,female,26.0000,0,0,7.9250,S,woman,False,N,True
3,1,1,female,35.0000,1,0,53.1000,S,woman,False,C,False
4,0,3,male,35.0000,0,0,8.0500,S,man,True,N,True


In [8]:
## 値 Label の数値化

In [9]:
# 辞書 mf_map の定義
mf_map = {'male': 1, 'female': 0}

# map 関数を利用して数値化
df7 = df6.copy()
df7['性別'] = df7['性別'].map(mf_map)

# 結果確認
display(df7.head())

,生存,等室,性別,年齢,兄弟配偶者数,両親子供数,料金,乗船港 Code,男女子供,成人男子,Deck,独身
0,0,3,1,22.0000,1,0,7.2500,S,man,True,N,False
1,1,1,0,38.0000,1,0,71.2833,C,woman,False,C,False
2,1,3,0,26.0000,0,0,7.9250,S,woman,False,N,True
3,1,1,0,35.0000,1,0,53.1000,S,woman,False,C,False
4,0,3,1,35.0000,0,0,8.0500,S,man,True,N,True


In [10]:
# 辞書 tf_map の定義
tf_map = {True: 1, False: 0}

# map 関数を利用して数値化
df8 = df7.copy()
df8['成人男子'] = df8['成人男子'].map(tf_map)

# map関数を利用して数値化
df9 = df8.copy()
df9['独身'] = df8['独身'].map(tf_map)

# 結果確認
display(df9.head())

,生存,等室,性別,年齢,兄弟配偶者数,両親子供数,料金,乗船港 Code,男女子供,成人男子,Deck,独身
0,0,3,1,22.0000,1,0,7.2500,S,man,1,N,0
1,1,1,0,38.0000,1,0,71.2833,C,woman,0,C,0
2,1,3,0,26.0000,0,0,7.9250,S,woman,0,N,1
3,1,1,0,35.0000,1,0,53.1000,S,woman,0,C,0
4,0,3,1,35.0000,0,0,8.0500,S,man,1,N,1
